In [ ]:
from IPython.display import Markdown

import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup

from pprint import pprint

Takes processed dataset as input, it also contains raw HTML.

Prints the first record of sampled patient

In [ ]:
data = pd.read_csv('../../data/data_processed/data_processed.csv')
data.index = data.record_number.values

def view_note(record_number = None,print_extracted = False):
    # sampling unique mr
    sample_mr = np.random.choice(data.mr.values)
    
    #getting the first record for one patient
    i = data.index.values[data.mr == sample_mr][0]
    if record_number:
        i = record_number
    print(f'Record Number # {i}')
    
    if print_extracted:
        print("***"*20)
        print("Hisroty ::")
        print(data.History.loc[i])
        print("\n")
        print("***"*20)
        print("Physical Exam ::")
        print(data.Physical_Exam.loc[i])
        print("\n")
        print("***"*20)
        print("Diagnoses ::")
        print(data.Diagnoses.loc[i])
        print("\n")
        print("***"*20)
        print("Problems ::")
        print(data.Problems.loc[i])
        print("\n")
        print("***"*20)
        print("Diagnostic_Test ::")
        print(data.Diagnostic_Test.loc[i])
        print("\n")
        print("***"*20)
        print("Cleaned Text ::")
        print(data.Cleaned_Text.loc[i])
        print("\n")
        print(f'Record Number # {i}')
        
        
    return Markdown(f'{data.comment_text[i]}')

In [ ]:
view_note()